In [47]:
%matplotlib inline
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3
import os
import cv2
import numpy as np
import torch
from torch.autograd import Variable
import quat_math
import pickle

from PIL import Image
import scipy.io as scio
from functools import partial
from object_pose_utils.utils import to_np, to_var

import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = 20, 12
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import warnings
warnings.filterwarnings('ignore')

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [48]:
from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset

from object_pose_utils.datasets.image_processing import ImageNormalizer
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

dataset_root = '/ssd0/datasets/ycb/YCB_Video_Dataset'
object_list = list(range(1,22))
mode = "test"

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION, 
                 otypes.IMAGE_CROPPED, 
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES]



In [49]:
from dense_fusion.network import PoseNetBingham
model_checkpoint = '/home/bokorn/src/DenseFusion/trained_models/ycb_global_bing/pose_model_1_4717.33.pth'

estimator = PoseNetBingham(num_points = 1000, num_obj = 21)
estimator.load_state_dict(torch.load(model_checkpoint, map_location=lambda storage, loc: storage))
estimator.cuda();

In [51]:
from generic_pose.datasets.ycb_dataset import getYCBSymmeties
from object_pose_utils.utils.pose_processing import symmetricAngularDistance, meanShift
from object_pose_utils.datasets.inplane_rotation_augmentation import InplaneRotator

import pathlib


import scipy.io as scio
from tqdm import tqdm_notebook as tqdm

err_bing_reg = {}
lik_bing_reg = {}

with torch.no_grad():
    for object_id in tqdm(object_list):
        sym_axis, sym_ang = getYCBSymmeties(object_id)
        ycb_dataset = YCBDataset(dataset_root, mode=mode, 
                                 object_list = [object_id], 
                                 output_data = output_format, 
                                 preprocessors = [InplaneRotator(np.pi/2)],
                                 postprocessors = [ImageNormalizer()],
                                 resample_on_error = False,
                                 image_size = [640, 480], num_points=1000)

        lik_bing_reg[object_id] = []
        err_bing_reg[object_id] = []
        
        pathlib.Path('results/data'.format(ycb_dataset.classes[object_id])).mkdir(parents=True, exist_ok=True)
        
        for j, data in tqdm(enumerate(ycb_dataset), total=len(ycb_dataset)):
            obj, quat, img, points, choose, = data
            if(len(obj) == 0):
                lik_bing_reg[object_id].append(np.nan)
                err_bing_reg[object_id].append(np.nan)
                continue
            
            idx = obj - 1
            points, choose, img, idx = Variable(points.unsqueeze(0)).cuda(), \
                                       Variable(choose.unsqueeze(0)).cuda(), \
                                       Variable(img.unsqueeze(0)).cuda(), \
                                       Variable(idx.unsqueeze(0)).cuda()
            
            pred_mean, pred_sigma, emb = estimator(img, points, choose, idx)
            pred_mean = pred_mean.flatten()
            pred_mean /= pred_mean.norm()
            pred_sigma = torch.abs(pred_sigma[:,:,0].view(-1))

                        
            lik_bing_reg[object_id].append(isobingham_likelihood(pred_mean, 
                                           pred_sigma,
                                           quat.unsqueeze(0).cuda()).item()*2)
            err_bing_reg[object_id].append(symmetricAngularDistance(pred_mean.cpu().unsqueeze(0), 
                                               quat.unsqueeze(0),
                                               sym_axis, sym_ang).item()*180/np.pi)

            savepath = 'results/{}'.format(ycb_dataset.getPath(j))
            pathlib.Path(savepath).mkdir(parents=True, exist_ok=True)
            
            scio.savemat('{}/{}_bing_reg_results_ROT.mat'.format(savepath, 
                         ycb_dataset.classes[object_id]),
                        {'index':j,
                         'quat':to_np(quat),
                         'pred_mean':to_np(pred_mean),
                         'pred_sigma':to_np(pred_sigma),
                         'lik_bing_reg':lik_bing_reg,
                         'err_bing_reg':err_bing_reg,
                         })

Exception on index 49: Mask 0050/000633 has less than minimum number of pixels (49 < 50)
Exception on index 50: Mask 0050/000640 has less than minimum number of pixels (41 < 50)
Exception on index 51: Mask 0050/000642 has less than minimum number of pixels (45 < 50)
Exception on index 52: Mask 0050/000644 has less than minimum number of pixels (18 < 50)
Exception on index 54: Mask 0050/000653 has less than minimum number of pixels (31 < 50)
Exception on index 55: Mask 0050/000660 has less than minimum number of pixels (16 < 50)
Exception on index 57: Mask 0050/000671 has less than minimum number of pixels (31 < 50)
Exception on index 58: Mask 0050/000681 has less than minimum number of pixels (7 < 50)
Exception on index 59: Mask 0050/000682 has less than minimum number of pixels (2 < 50)
Exception on index 60: Mask 0050/000701 has less than minimum number of pixels (41 < 50)
Exception on index 553: No points in mask
Exception on index 559: No points in mask
Exception on index 560: No p

Exception on index 87: Mask 0049/000833 has less than minimum number of pixels (47 < 50)
Exception on index 108: Mask 0049/001157 has less than minimum number of pixels (2 < 50)
Exception on index 111: Mask 0049/001179 has less than minimum number of pixels (47 < 50)
Exception on index 115: Mask 0049/001222 has less than minimum number of pixels (13 < 50)
Exception on index 116: Mask 0049/001229 has less than minimum number of pixels (10 < 50)
Exception on index 118: Mask 0049/001247 has less than minimum number of pixels (50 < 50)
Exception on index 129: Mask 0049/001340 has less than minimum number of pixels (40 < 50)
Exception on index 138: Mask 0049/001441 has less than minimum number of pixels (25 < 50)
Exception on index 322: No points in mask
Exception on index 323: No points in mask
Exception on index 324: No points in mask
Exception on index 325: No points in mask
Exception on index 326: No points in mask
Exception on index 327: No points in mask
Exception on index 334: No poi

Exception on index 659: No points in mask
Exception on index 661: No points in mask
Exception on index 665: No points in mask
Exception on index 666: No points in mask
Exception on index 671: No points in mask
Exception on index 672: No points in mask
Exception on index 673: No points in mask
Exception on index 674: No points in mask


Exception on index 174: No points in mask
Exception on index 175: No points in mask
Exception on index 176: No points in mask
Exception on index 177: No points in mask
Exception on index 178: No points in mask
Exception on index 179: No points in mask
Exception on index 180: No points in mask


In [52]:
np.savez('results/df_bingreg_ROT.npz', 
         err_reg=err_bing_reg,
         lik_reg=lik_bing_reg,
         )

In [54]:
list(err_bing_reg[9])

[132.5553390338917,
 128.57426765350138,
 122.69275077806839,
 107.85428774962075,
 101.0939099431147,
 91.86463048751331,
 93.85848564964509,
 81.55791601109908,
 83.16238208864844,
 93.91636467267178,
 91.81170335163488,
 91.0132952187954,
 95.3072370744965,
 91.86027282682284,
 97.74241807965949,
 96.47559557354053,
 94.2871005106309,
 94.34898885470038,
 93.97843060165968,
 94.09179808150355,
 94.6107831753978,
 94.5315188300798,
 95.0797166430542,
 95.96456081984019,
 94.92503334892092,
 101.69306096729738,
 103.39261010828248,
 100.76314437217849,
 101.1831258740534,
 99.25834441499796,
 100.10196840014292,
 101.03951431656472,
 102.66646537686175,
 104.23364545173202,
 108.07917172804342,
 102.21287251408121,
 101.27320240882753,
 99.09304700689448,
 97.12510558247372,
 96.68539166408665,
 95.33845786919262,
 94.25921967843891,
 92.33848169137211,
 96.47437979986827,
 95.90800685351248,
 95.19382178332843,
 104.87380493169145,
 177.46431093996745,
 147.76387537194898,
 108.03721

In [32]:

pred_r, pred_t, pred_c, emb = estimator_base(img, points, choose, idx)
pred_r = pred_r[0,torch.argmax(pred_c)][[1,2,3,0]]
pred_r /= pred_r.norm()

In [24]:
from object_pose_utils.utils.bingham import isobingham_likelihood

print(pred_mean)
print(pred_sigma)
print(quat)

tensor([ 0.0078, -0.7962, -0.4496,  0.4047], device='cuda:0',
       grad_fn=<AsStridedBackward>)
tensor([3.7406], device='cuda:0', grad_fn=<AbsBackward>)
tensor([ 0.6979, -0.3533,  0.2597,  0.5663])


In [35]:
print(symmetricAngularDistance(pred_mean.unsqueeze(0), quat.unsqueeze(0).cuda(),
                         sym_axis, sym_ang).item()*180/np.pi)
print(isobingham_likelihood(pred_mean, pred_sigma, quat.unsqueeze(0).cuda()))

132.96025996864674
tensor([[0.0209]], device='cuda:0', grad_fn=<MulBackward0>)


In [34]:
print(symmetricAngularDistance(pred_r.unsqueeze(0), quat.unsqueeze(0).cuda(),
                         sym_axis, sym_ang).item()*180/np.pi)
print(torch.log(isobingham_likelihood(pred_r, pred_sigma, quat.unsqueeze(0).cuda())))

9.657612571283936
tensor([[-0.7509]], device='cuda:0', grad_fn=<LogBackward>)


In [1]:
from object_pose_utils.utils.bingham import makeDuelMatrix, makeDuelMatrix2
import torch
q = torch.Tensor([1,2,3,4]).float().cuda()
q = q/q.norm()

makeDuelMatrix2(q)
a, b, c, d = q
zero = torch.tensor(0).float()
zero = zero.cuda()

torch.stack([zero, a*a+b*b-c*c-d*d,   2*(b*c - a*d),   2*(b*d + a*c)])


tensor([ 0.0000, -0.6667,  0.1333,  0.7333], device='cuda:0')

In [16]:
torch.tensor(0)

tensor(0)